In [ ]:
import sys
import os
sys.path.insert(0, "../../python")

import numpy as np
from beam.beams import electronbeam
from beam.elements import plasma
import matplotlib.pyplot as plt
from beam import interactions
from ionization import ionization
from lens import profile
plt.rcParams['animation.ffmpeg_path'] = '/home/robert/anaconda3/envs/CU-PWFA/bin/ffmpeg'
import matplotlib.animation as animation
from scipy.optimize import curve_fit

path = '/media/robert/Data_Storage/Data/Plots/BeamEvolution/'

In [ ]:
%npl0   = 1e16 # cm^-3
%dEds0  = np.sqrt(npl0/(1e17))*10e9 # eV/m
%dgds0  = dEds0/0.511e6
L_ft   = 0.50 # m

# define plasma up-ramp
hw_up    = 0.1205 #0.05 # m
L_up     = 1.5 # m
top_up   = L_up # m

# define plasma down-ramp
shape_dn = shape_up
hw_dn    = hw_up # m
L_dn     = L_up # m
top_dn   = 0  # m

# define beam parameters
%gbC    = 20000 # relativistic lorentz factor
%eps    = 5.0e-6  # m-rad, normalized emittance
%beta   = 0.10  # m

alpha  = 0.00
gamma  = (1.0+alpha**2)/beta # 1/m
dgb    = 0.01
dz     = 0
%npart  = 10000

%s_w   = L_up + waist

In [ ]:
# Now we can test my code and see if it is different
z = np.linspace(0, 4e6, 10000)
ne = 
betaBeam = beta+(-s_w)**2/beta
alphaBeam = s_w/beta
electronParams = {
    'name' : 'TestBeam',
    'path' : path,
    'load' : False,
    'N' : npart,
    'gamma' : gbC,
    'emittance' : eps,
    'betax' : betaBeam,
    'betay' : betaBeam,
    'alphax' : alphaBeam,
    'alphay' : alphaBeam,
    'sigmaz' : 5e-6,
    'dE' : 0.01
}

beam = electronbeam.GaussianElectronBeam(electronParams)
beam.plot_current_phase()

class plasmaCls():
    def __init__(self):
        """ Empy init function """
    def get_ne(self, z):
        return ne
    def dgammadz(self, ne):
        npl = ne*1e17
        if npl > ((4/9)*npl0):
            dgdz = dgds0*np.sqrt(npl/npl0)*(3*np.sqrt(npl/npl0)-2)
        else:
            dgdz = (-dgds0)*(2/np.pi)*np.sqrt(npl/npl0)*np.sin((3*np.pi/2)*np.sqrt(npl/npl0))
        return dgdz*1e-6
    
plasmaDen = plasmaCls()
interactions.electron_plasma(beam, plasmaDen, z, 1, 4)

In [ ]:
# Now time to animate phase space
ptcls, z = beam.load_ptcls(0)
fig, sctx, scty = beam.plot_phase(ptcls, z, )#xlim=[-400, 400])
i = 1
def updatefig(*args):
    global i
    ptcls = beam.load_ptcls(i)[0]
    sctx.set_offsets(np.stack((ptcls[:, 0]*1e6, ptcls[:, 1]*1e3), axis=-1))
    scty.set_offsets(np.stack((ptcls[:, 2]*1e6, ptcls[:, 3]*1e3), axis=-1))
    i += 1
    return sctx, scty,
ani = animation.FuncAnimation(fig, updatefig, blit=True, frames=m-3)
ani.save(path + 'PhaseSpaceEvolution.mp4', fps=30)